# notes

all using pixelwise loss and equivalent scheduler params 

In [1]:
# make a cell print all the outputs instead of just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from pathlib import Path

data_dir = Path("./data")
data_fpath = data_dir / "2022-07-bmvc-report-01-data04-sweep-04-swa-00-partial-00.csv"

import pandas as pd
df = pd.read_csv(data_fpath) 

def check_df(df):
    print(df.shape)
    print(df.dtypes)
    return df.head(5)

check_df(df)

(223, 11)
Name                            object
normal_class                     int64
supervise_mode                  object
lightning_swa_epoch_start      float64
lightning_swa_learning_rate    float64
State                           object
mvtec_class_type                object
learning_rate                  float64
seed_str                        object
test/avg-precision             float64
test/roc-auc                   float64
dtype: object


,Name,normal_class,supervise_mode,lightning_swa_epoch_start,lightning_swa_learning_rate,State,mvtec_class_type,learning_rate,seed_str,test/avg-precision,test/roc-auc
0,deft-sweep-303,5,synthetic-anomaly-confetti,0.2,0.01,finished,object,0.001,0x40452d6f87068d81,0.067131,0.806190
1,faithful-sweep-301,5,synthetic-anomaly-confetti,0.2,0.01,finished,object,0.001,0x526f289f74732498,0.070996,0.822657
2,trim-sweep-302,5,real-anomaly,0.2,0.01,finished,object,0.001,0x526f289f74732498,0.372534,0.972140
3,whole-sweep-304,5,real-anomaly,0.2,0.01,finished,object,0.001,0x40452d6f87068d81,0.424343,0.975034
4,royal-sweep-279,4,synthetic-anomaly-confetti,0.5,0.01,finished,texture,0.001,0xeb33f3d140c7b5f4,0.126786,0.864353


In [3]:
data_df = pd.read_csv(data_fpath) 
df = data_df
df.drop(columns=["State", "Name",], inplace=True)
check_df(df)

(223, 9)
normal_class                     int64
supervise_mode                  object
lightning_swa_epoch_start      float64
lightning_swa_learning_rate    float64
mvtec_class_type                object
learning_rate                  float64
seed_str                        object
test/avg-precision             float64
test/roc-auc                   float64
dtype: object


,normal_class,supervise_mode,lightning_swa_epoch_start,lightning_swa_learning_rate,mvtec_class_type,learning_rate,seed_str,test/avg-precision,test/roc-auc
0,5,synthetic-anomaly-confetti,0.2,0.01,object,0.001,0x40452d6f87068d81,0.067131,0.806190
1,5,synthetic-anomaly-confetti,0.2,0.01,object,0.001,0x526f289f74732498,0.070996,0.822657
2,5,real-anomaly,0.2,0.01,object,0.001,0x526f289f74732498,0.372534,0.972140
3,5,real-anomaly,0.2,0.01,object,0.001,0x40452d6f87068d81,0.424343,0.975034
4,4,synthetic-anomaly-confetti,0.5,0.01,texture,0.001,0xeb33f3d140c7b5f4,0.126786,0.864353


In [4]:
data_df = df

In [5]:
from pandas import DataFrame
import pandas as pd
import numpy as np
from scipy.stats import percentileofscore
import matplotlib as mpl
import matplotlib.cm as cm

def get_color_mask_compare_by_line_every_n_cols(pt, n):
        
    assert pt.shape[1] % n == 0, f"pt.shape[1] % n != 0: {pt.shape[1] % n}"

    def get_ordering_signal(subdf: DataFrame) -> DataFrame:
        """at each line, the values are assigned a value from 0 to 1 respecting the ordering, such that 0 is the min and 1 is the max"""
        # it's ok to round up to 3 decimals because i will probably never have more than 1001 values
        # and it makes the table easier to read/debug
        
        if subdf.isna().any(axis=None):
            return DataFrame(-1 * np.ones_like(subdf.values), columns=subdf.columns, index=subdf.index).astype(int)
        
        subdf = subdf.apply(
            lambda series: series.apply(lambda x: percentileofscore(np.sort(series.values), x, kind="strict") / 100), 
            axis=1
        )
        # 1/n is the min value of percentileofscore
        return (subdf / (1 / n)).round(decimals=0).astype(int)

    return np.concatenate([
        np.concatenate(
            [   
                get_ordering_signal(pt.iloc[rowidx:(rowidx + 1), i * n:(i + 1) * n]).values
                for i in range(pt.shape[1] // n)
            ],
            axis=1,
        )
        for rowidx in range(pt.shape[0])
    ], axis=0)
    
COLOR_MINUS_1 = "background-color: #00000000"
alpha = "55"
COLOR_0 = f'background-color: #DC143C{alpha}'
COLOR_MIDDLE = f'background-color: #FFFFFF{alpha}'
COLOR_1 = f'background-color: #228B22{alpha}'

def color_from_mask(color_mask: np.ndarray):
    n = color_mask.max() + 1
    assert n > 1, f"n > 1: {n}"
    
    if n == 2:
        def color(_):
            colormap = np.empty_like(color_mask, dtype=object)
            colormap[color_mask == -1] = COLOR_MINUS_1
            colormap[color_mask == 0] = COLOR_0
            colormap[color_mask == 1] = COLOR_1
            return colormap    
        return color
    
    elif n == 3:
        def color(_):
            colormap = np.empty_like(color_mask, dtype=object)
            colormap[color_mask == -1] = COLOR_MINUS_1
            colormap[color_mask == 0] = COLOR_0
            colormap[color_mask == 1] = COLOR_MIDDLE
            colormap[color_mask == 2] = COLOR_1
            return colormap
        return color
    
    elif n >= 4:
        def color(_):
            colormap = np.empty_like(color_mask, dtype=object)
            colormap[color_mask == -1] = COLOR_MINUS_1
            
            to_rgba = cm.ScalarMappable(
                norm=mpl.colors.Normalize(vmin=0, vmax=(n-1)), 
                cmap=cm.get_cmap("bwr_r")
            ).to_rgba
            
            def rgba2str(rgba):
                rgba = (255 * np.array(rgba)).astype(int)
                return f"background-color: #{rgba[0]:02x}{rgba[1]:02x}{rgba[2]:02x}{alpha}"

            import itertools
            size_i, size_j = color_mask.shape
            
            # i got lazy and used loops
            for idxi, idxj in itertools.product(range(size_i), range(size_j)):
                val = color_mask[idxi, idxj]
                if val == -1:
                    colormap[idxi, idxj] = COLOR_MINUS_1
                    continue
                colormap[idxi, idxj] = rgba2str(to_rgba(val))
            return colormap
        return color
    else:
        raise NotImplementedError(f"n > 3 is not implemented, found {n}")

In [6]:
from pandas import DataFrame
import pandas as pd
import numpy as np
from functools import partial

df = data_df.copy()
	
row_cols = ["mvtec_class_type", "normal_class"]
column_cols = ["supervise_mode", "lightning_swa_epoch_start", "lightning_swa_learning_rate"]
	
metrics = {
    "test/avg-precision": ["mean", "std"],
    # "validate/avg-precision-histavg": ["mean",],
    # "test/roc-auc": ["mean", "std"],
    # "validate/roc-auc-histavg": ["mean",],
}
metric_cols = [(k, v) for k, list_vals in metrics.items() for v in list_vals]

pt_all_metrics = pd.pivot_table(df, index=row_cols, columns=column_cols, aggfunc=metrics)

# when it's a percentage
pt_all_metrics = 100 * pt_all_metrics

pt_test_avg = pt_all_metrics[("test/avg-precision", "mean")]
# pt_test_avg = pt_all_metrics[("test/roc-auc", "mean")]

pt_test_avg.loc[("", "object mean"), :] = pt_test_avg.loc["object"].mean(axis=0)
pt_test_avg.loc[("", "texture mean"), :] = pt_test_avg.loc["texture"].mean(axis=0)
pt_test_avg.loc[("", "all mean"), :] = pt_test_avg.mean(axis=0)

pt_test_std = pt_all_metrics[("test/avg-precision", "std")]
# # pt_test_std = pt_all_metrics[("test/roc-auc", "std")]

pt_test_std.loc[("", "object mean"), :] = pt_test_std[~pt_test_std.isna()].loc["object"].mean(axis=0)
pt_test_std.loc[("", "texture mean"), :] = pt_test_std[~pt_test_std.isna()].loc["texture"].mean(axis=0)
pt_test_std.loc[("", "all mean"), :] = pt_test_std[~pt_test_std.isna()].mean(axis=0)

# pt_validate_histmean_avg = pt_all_metrics[("validate/avg-precision-histavg", "mean")]
# pt_validate_histmean_avg = pt_all_metrics[("validate/roc-auc-histavg", "mean")]

# pt_validate_histmean_avg.loc[("", "object mean"), :] = pt_validate_histmean_avg.loc["object"].mean(axis=0)
# pt_validate_histmean_avg.loc[("", "texture mean"), :] = pt_validate_histmean_avg.loc["texture"].mean(axis=0)
# pt_validate_histmean_avg.loc[("", "all mean"), :] = pt_validate_histmean_avg.mean(axis=0)

pt_composed = pt_test_avg.applymap("{:.1f}".format) + " " + pt_test_std.applymap("({:.1f})".format) # + " " + pt_validate_histmean_avg.applymap("[{:.1f}]".format)
pt_composed

/tmp/ipykernel_65677/2676871268.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pt_test_avg.loc[("", "object mean"), :] = pt_test_avg.loc["object"].mean(axis=0)
/tmp/ipykernel_65677/2676871268.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pt_test_std.loc[("", "object mean"), :] = pt_test_std[~pt_test_std.isna()].loc["object"].mean(axis=0)


supervise_mode                real-anomaly                          \
lightning_swa_epoch_start              0.2                           
lightning_swa_learning_rate          0.001       0.003       0.010   
mvtec_class_type normal_class                                        
object           0              62.9 (8.3)  66.2 (7.1)  64.3 (6.9)   
                 1              60.8 (2.6)  60.1 (3.0)  59.2 (3.6)   
                 2              13.6 (4.6)  12.9 (4.2)  10.5 (2.8)   
                 5               nan (nan)   nan (nan)  39.8 (3.7)   
texture          3              72.5 (3.0)  69.4 (4.4)  63.5 (3.1)   
                 4              20.6 (0.8)  17.6 (1.4)  16.8 (2.0)   
                 object mean    45.8 (5.2)  46.4 (4.8)  43.5 (4.2)   
                 texture mean   46.6 (1.9)  43.5 (2.9)  40.2 (2.6)   
                 all mean       46.1 (3.8)  45.1 (4.0)  42.2 (3.6)   

supervise_mode                                                      \
lightning_swa_epoch_start              0.5                           
lightning_swa_learning_rate          0.001       0.003       0.010   
mvtec_class_type normal_class                                        
object           0             67.1 (10.1)  65.2 (8.0)  65.5 (8.6)   
                 1              60.7 (2.9)  60.0 (3.0)  61.3 (4.6)   
                 2              12.7 (5.2)  12.8 (3.6)  12.4 (4.1)   
                 5               nan (nan)   nan (nan)   nan (nan)   
texture          3              71.2 (0.8)   nan (nan)  66.7 (nan)   
                 4              20.4 (1.2)  18.6 (1.9)  17.3 (1.7)   
                 object mean    46.8 (6.0)  46.0 (4.8)  46.4 (5.8)   
                 texture mean   45.8 (1.0)  18.6 (1.9)  42.0 (1.7)   
                 all mean       46.4 (3.9)  36.9 (3.8)  44.5 (4.4)   

supervise_mode                synthetic-anomaly-confetti              \
lightning_swa_epoch_start                            0.2               
lightning_swa_learning_rate                        0.001       0.003   
mvtec_class_type normal_class                                          
object           0                            19.3 (1.0)  17.1 (0.4)   
                 1                            54.1 (1.8)  54.7 (1.3)   
                 2                             9.4 (1.0)   9.0 (1.1)   
                 5                             nan (nan)   nan (nan)   
texture          3                            43.2 (2.3)  40.5 (5.4)   
                 4                            12.6 (0.4)  13.3 (0.4)   
                 object mean                  27.6 (1.3)  26.9 (0.9)   
                 texture mean                 27.9 (1.3)  26.9 (2.9)   
                 all mean                     27.7 (1.3)  26.9 (1.8)   

supervise_mode                                                                 
lightning_swa_epoch_start                         0.5                          
lightning_swa_learning_rate         0.010       0.001       0.003       0.010  
mvtec_class_type normal_class                                                  
object           0             16.0 (0.1)  18.1 (0.7)  17.2 (0.1)  16.3 (0.2)  
                 1             54.5 (0.7)  54.6 (1.7)  54.3 (1.6)  54.3 (1.3)  
                 2              7.7 (0.9)   9.0 (0.7)   8.2 (0.3)   8.0 (1.0)  
                 5              6.9 (0.3)   nan (nan)   nan (nan)   nan (nan)  
texture          3             37.5 (3.6)  41.2 (5.0)  39.5 (4.8)  32.0 (nan)  
                 4             12.1 (0.6)  13.5 (0.5)  13.4 (0.4)  12.8 (0.2)  
                 object mean   21.3 (0.5)  27.2 (1.1)  26.6 (0.7)  26.2 (0.8)  
                 texture mean  24.8 (2.1)  27.3 (2.7)  26.5 (2.6)  22.4 (0.2)  
                 all mean      22.6 (1.1)  27.3 (1.8)  26.5 (1.5)  24.6 (0.6)

# all: within same setup (`(epoch_start, learning_rate)` value compared)

In [7]:
pt = pt_composed.copy()
pt_color_from = pt_test_avg.copy()

color_mask = get_color_mask_compare_by_line_every_n_cols(pt_color_from, n=6)
pt.style.apply(color_from_mask(color_mask), axis=None)

# all: within same setup + `epoch_start` (`learning_rate` value compared)

In [8]:
pt = pt_composed.copy()
pt_color_from = pt_test_avg.copy()

color_mask = get_color_mask_compare_by_line_every_n_cols(pt_color_from, n=3)
pt.style.apply(color_from_mask(color_mask), axis=None)

lowest learning rate (`1e-3`) seems to always be the best 

# all: within same setup + `learning_rate` (`epoch_start` value compared)

In [9]:
neworder = [0, 2, 1]

def reorder_columns(df_):
    level_names = np.array(df_.columns.names)[neworder].tolist()
    df_ = df_.reorder_levels(neworder, axis=1)
    df_.columns = pd.MultiIndex.from_tuples(np.sort(df_.columns.values), names=level_names)
    return df_

pt = reorder_columns(pt_composed.copy())
pt_color_from = reorder_columns(pt_test_avg.copy())

color_mask = get_color_mask_compare_by_line_every_n_cols(pt_color_from, n=2)
pt.style.apply(color_from_mask(color_mask), axis=None)

it looks like the lr were too high

when too high the later start of swa is doing better

when we choose the best lr overall, then the best start is the earliest